In [1]:
from genetic_algorithm import *
from numpy.random import default_rng
import matplotlib.pyplot as plt

In [2]:

# ## hexadecimal generator

nr_of_input = 3
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
world_size_x = 20
world_size_y = 20

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)

# ## initial brain and position generator

result = calculate_individual_output_weights(individuals)
print(result)



{0: {'out4': 0.34, 'out1': 0.043, 'out2': 0.113, 'out0': 0.981}, 1: {'out1': 0.005, 'out0': 0.995, 'out3': 0.859}, 2: {'out3': 0.658, 'out2': 0.618, 'out4': 0.021}, 3: {'out1': 0.75, 'out2': 0.001, 'out4': 0.02, 'out0': 0.855, 'out3': 0.118}}


In [3]:
for indiv in result:
    result[indiv]['position'] = [[x[indiv], y[indiv]]]

In [4]:
result

{0: {'out4': 0.34,
  'out1': 0.043,
  'out2': 0.113,
  'out0': 0.981,
  'position': [[4, 8]]},
 1: {'out1': 0.005, 'out0': 0.995, 'out3': 0.859, 'position': [[6, 5]]},
 2: {'out3': 0.658, 'out2': 0.618, 'out4': 0.021, 'position': [[19, 7]]},
 3: {'out1': 0.75,
  'out2': 0.001,
  'out4': 0.02,
  'out0': 0.855,
  'out3': 0.118,
  'position': [[11, 6]]}}

In [5]:
k = 15
n = 0
while k>n:  
    for indiv in result:
        x = result[indiv]['position'][-1][0]
        y = result[indiv]['position'][-1][1]
        position_list = []
        for out in result[indiv]:
            if 'out' in out:
                new_pos = move(out, result[indiv][out])

                position_list.append(new_pos)
        
        position_list = list(map(sum, zip(*position_list)))
#         print(position_list)
        position_list = make_smaller_(position_list)
#         print(position_list)
        position_list = list(map(sum, zip(*[[x, y]] + [position_list])))

        position_list[0] = normalize_position_if_outside_world(position_list[0], world_size_x)
        position_list[1] = normalize_position_if_outside_world(position_list[1], world_size_y)
        
        
        result[indiv]['position'].append(position_list)
    n += 1
            
result

new_pos [-1, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 1]
new_pos [-1, 0]
new_pos [0, 0]
new_pos [1, 0]
new_pos [0, 0]
new_pos [0, -1]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 1]
new_pos [-1, 0]
new_pos [-1, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, -1]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, -1]
new_pos [0, 0]
new_pos [1, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 1]
new_pos [-1, 0]
new_pos [-1, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, -1]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [-1, 0]
new_pos [0, 0]
new_pos [0, 0]
new_pos [0, 1]
new_pos [0, 0]
new_pos [0, 1]

{0: {'out4': 0.34,
  'out1': 0.043,
  'out2': 0.113,
  'out0': 0.981,
  'position': [[4, 8],
   [3, 9],
   [3, 10],
   [3, 11],
   [4, 11],
   [3, 12],
   [2, 12],
   [2, 13],
   [2, 13],
   [1, 13],
   [1, 14],
   [1, 15],
   [0, 16],
   [0, 17],
   [0, 18],
   [0, 19]]},
 1: {'out1': 0.005,
  'out0': 0.995,
  'out3': 0.859,
  'position': [[6, 5],
   [6, 6],
   [5, 7],
   [4, 8],
   [3, 9],
   [2, 10],
   [1, 11],
   [0, 12],
   [0, 13],
   [0, 14],
   [0, 15],
   [0, 16],
   [0, 17],
   [0, 18],
   [0, 19],
   [0, 20]]},
 2: {'out3': 0.658,
  'out2': 0.618,
  'out4': 0.021,
  'position': [[19, 7],
   [19, 7],
   [20, 7],
   [19, 7],
   [18, 7],
   [18, 7],
   [18, 7],
   [17, 7],
   [18, 7],
   [19, 7],
   [18, 7],
   [19, 7],
   [18, 7],
   [18, 7],
   [19, 7],
   [19, 7]]},
 3: {'out1': 0.75,
  'out2': 0.001,
  'out4': 0.02,
  'out0': 0.855,
  'out3': 0.118,
  'position': [[11, 6],
   [11, 7],
   [11, 7],
   [11, 7],
   [11, 7],
   [11, 7],
   [11, 6],
   [11, 7],
   [11, 8],
   [1

In [22]:
[9, 8]==[9, 8]

True

In [7]:
a = [(3, 0),(4,1),(5,2)]

def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

# print(slope_intercept(3,0,2,1))

if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
# print(x)

a[-2][0]



4